
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>



## Hourly Activity by Traffic Lab
Process streaming data to display the total active users by traffic source with a 1 hour window.
1. Cast to timestamp and add watermark for 2 hours
2. Aggregate active users by traffic source for 1 hour windows
3. Execute query with **`display`** and plot results
5. Use query name to stop streaming query

(pt-br)

## Atividade por hora do Traffic Lab
Processe dados de streaming para exibir o total de usuários ativos por origem de tráfego com uma janela de 1 hora.
1. Transmita para carimbo de data/hora e adicione marca d'água por 2 horas
2. Agregue usuários ativos por origem de tráfego em janelas de 1 hora
3. Execute a consulta com **`display`** e plote os resultados
5. Use o nome da consulta para interromper a consulta de streaming



### Setup
Run the cells below to generate hourly JSON files of event data for July 3, 2020.

In [0]:
%run ../Includes/Classroom-Setup-5.1b

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| removing the working directory "dbfs:/mnt/dbacademy-users/anacadriano20@gmail.com/apache-spark-programming-with-databricks"...(0 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03"

Validating the locally installed datasets:
| listing local files...(4 seconds)
| validation completed...(4 seconds total)

Creating & using the schema "anacadriano20_6ryf_da_asp" in the catalog "spark_catalog"...(2 seconds)

Predefined tables in "anacadriano20_6ryf_da_asp":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/anacadriano20@gmail.com/apache-spark-programming-with-databricks
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/anacadriano20@gmail.com/apache-spark-programming-with-databricks/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03
| DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/an

In [0]:
schema = "device STRING, ecommerce STRUCT<purchase_revenue_in_usd: DOUBLE, total_item_quantity: BIGINT, unique_items: BIGINT>, event_name STRING, event_previous_timestamp BIGINT, event_timestamp BIGINT, geo STRUCT<city: STRING, state: STRING>, items ARRAY<STRUCT<coupon: STRING, item_id: STRING, item_name: STRING, item_revenue_in_usd: DOUBLE, price_in_usd: DOUBLE, quantity: BIGINT>>, traffic_source STRING, user_first_touch_timestamp BIGINT, user_id STRING"

# Directory of hourly events logged from the BedBricks website on July 3, 2020
hourly_events_path = f"{DA.paths.datasets}/ecommerce/events/events-2020-07-03.json"

df = (spark.readStream
           .schema(schema)
           .option("maxFilesPerTrigger", 1)
           .json(hourly_events_path))



### 1. Cast to timestamp and add watermark for 2 hours
- Add a **`createdAt`** column by dividing **`event_timestamp`** by 1M and casting to timestamp
- Set a watermark of 2 hours on the **`createdAt`** column

Assign the resulting DataFrame to **`events_df`**.

In [0]:
# ANSWER
from pyspark.sql.functions import col

events_df = (df.withColumn("createdAt", (col("event_timestamp") / 1e6).cast("timestamp"))
               .withWatermark("createdAt", "2 hours"))

In [0]:
display(events_df)

java.net.NoRouteToHostException: No route to host
	at java.base/sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at java.base/sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:777)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:347)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1700(ManagedSelector.java:65)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:676)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:535)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:362)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:186)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:173)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:409)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$7(InstrumentedQueuedThreadPool.scala:158)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads(QueuedThreadPoolInstrumenter.scala:66)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads$(QueuedThreadPoolInstrumenter.scala:63)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.trackActiveThreads(InstrumentedQueuedThreadPool.scala:124)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.run(InstrumentedQueuedThreadPool.scala:153)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:883)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:1034)
	at java.base/java.lang.Thread.run(Thread.java:829)




**1.1: CHECK YOUR WORK**

In [0]:
DA.tests.validate_1_1(events_df.schema)

Points,Test,Result
1,Schema is of type StructType,
1,Schema contians 12 field,
1,"Schema contains ""device"" of type StringType",
1,"Schema contains ""ecommerce"" of type StructType",
1,"Schema contains ""event_name"" of type StringType",
1,"Schema contains ""event_previous_timestamp"" of type LongType",
1,"Schema contains ""event_timestamp"" of type LongType",
1,"Schema contains ""geo"" of type StructType",
1,"Schema contains ""items"" of type ArrayType",
1,"Schema contains ""traffic_source"" of type StringType",




### 2. Aggregate active users by traffic source for 1 hour windows

- Set the default shuffle partitions to the number of cores on your cluster
- Group by **`traffic_source`** with 1-hour tumbling windows based on the **`createdAt`** column
- Aggregate the approximate count of distinct users per **`user_id`** and alias the resulting column to **`active_users`**
- Select **`traffic_source`**, **`active_users`**, and the **`hour`** extracted from **`window.start`** with an alias of **`hour`**
- Sort by **`hour`** in ascending order
Assign the resulting DataFrame to **`traffic_df`**.

In [0]:
# ANSWER
from pyspark.sql.functions import approx_count_distinct, hour, window

spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)

traffic_df = (events_df
              .groupBy("traffic_source", window(col("createdAt"), "1 hour"))
              .agg(approx_count_distinct("user_id").alias("active_users"))
              .select(col("traffic_source"), col("active_users"), hour(col("window.start")).alias("hour"))
              .sort("hour")
             )

In [0]:
display(traffic_df)

java.net.NoRouteToHostException: No route to host
	at java.base/sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at java.base/sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:777)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:347)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1700(ManagedSelector.java:65)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:676)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:535)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:362)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:186)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:173)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:409)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$7(InstrumentedQueuedThreadPool.scala:158)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads(QueuedThreadPoolInstrumenter.scala:66)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads$(QueuedThreadPoolInstrumenter.scala:63)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.trackActiveThreads(InstrumentedQueuedThreadPool.scala:124)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.run(InstrumentedQueuedThreadPool.scala:153)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:883)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:1034)
	at java.base/java.lang.Thread.run(Thread.java:829)




**2.1: CHECK YOUR WORK**

In [0]:
DA.tests.validate_2_1(traffic_df.schema)

Points,Test,Result
1,Schema is of type StructType,
1,Schema contians three field,
1,"Schema contains ""traffic_source"" of type StringType",
1,"Schema contains ""active_users"" of type LongType",
1,"Schema contains ""hour"" of type IntegerType",




### 3. Execute query with display() and plot results
- Use **`display`** to start **`traffic_df`** as a streaming query and display the resulting memory sink
  - Assign "hourly_traffic" as the name of the query by setting the **`streamName`** parameter of **`display`**
- Plot the streaming query results as a bar graph
- Configure the following plot options:
  - Keys: **`hour`**
  - Series groupings: **`traffic_source`**
  - Values: **`active_users`**

In [0]:
# ANSWER
display(traffic_df, streamName="hourly_traffic")

traffic_source,active_users,hour
youtube,479,0
facebook,1503,0
instagram,907,0
google,2523,0
email,787,0
direct,806,0





**3.1: CHECK YOUR WORK**

- The bar chart should plot **`hour`** on the x-axis and **`active_users`** on the y-axis
- Six bars should appear at every hour for all traffic sources
- The chart should stop at hour 23



### 4. Manage streaming query
- Iterate over SparkSession's list of active streams to find one with name "hourly_traffic"
- Stop the streaming query

In [0]:
# ANSWER
DA.block_until_stream_is_ready("hourly_traffic")

for s in spark.streams.active:
    if s.name == "hourly_traffic":
        s.stop()
        s.awaitTermination()

Processed 0 of 2 batches...
Processed 0 of 2 batches...
Processed 0 of 2 batches...
Processed 1 of 2 batches...
Processed 2 of 2 batches...
The stream is now active with 2 batches having been processed.





**4.1: CHECK YOUR WORK**
Print all active streams to check that "hourly_traffic" is no longer there

In [0]:
DA.tests.validate_4_1()

Points,Test,Result
1,All queries have stopped streaming,




### Classroom Cleanup
Run the cell below to clean up resources.

In [0]:
DA.cleanup()

Resetting the learning environment:
| dropping the schema "anacadriano20_6ryf_da_asp"...(1 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/anacadriano20@gmail.com/apache-spark-programming-with-databricks"...(1 seconds)

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)


&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>